In [3]:
###FEATURE EXTRACTION

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#Getting the data
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

IMAGE_SIZE = (224,224)
BATCH_SIZE = 64

train_ds = train_gen.flow_from_directory(train_dir,
                                         batch_size=BATCH_SIZE,
                                         target_size=IMAGE_SIZE,
                                         class_mode="categorical")
test_ds = test_gen.flow_from_directory(test_dir,
                                      batch_size=BATCH_SIZE,
                                      target_size=IMAGE_SIZE,
                                      class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [5]:
#callbacks
#tensorboard
import datetime

def create_tensorboard_callback(dir_name, exp_name):
    log_dir = dir_name + "/" + exp_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print("Saving TensorBoard log files to", logdir)
    return tensorboard_callback

In [11]:
#Creating models using tensorflow hub
import tensorflow_hub as hub
from tensorflow.keras import layers

def create_model(model_url, num_classes=10):
    """
    Takes a tf hub url and creates a Sequential model with it
    """
    feature_extractor_layer = hub.KerasLayer(model_url, 
                                             trainable=False,
                                            input_shape=IMAGE_SIZE+(3,))
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax")
    ])
    
    return model

In [12]:
efficientnet_v2_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2"

mobilenet_v3_url = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"

efficientnet_model = create_model(efficientnet_v2_url)

mobilenet_model = create_model(mobilenet_v3_url)

Metal device set to: Apple M1 Max


2023-09-06 19:38:19.889614: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-06 19:38:19.890006: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
efficientnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])